# Case Study 1: Educational Outcomes for Hearing-impaired Children

In [ ]:
# Import data analysis packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Import pymc modules
import arviz as av
import pymc as pm

Here, we are interested in determining factors associated with
better or poorer learning outcomes.

## The Data

The anonymized dataset is taken from the Listening and
Spoken Language Data Repository (LSL-DR), an international data repository.

The anonymized data set tracks the demographics and longitudinal
outcomes for children who have hearing loss and are enrolled in
programs focused on supporting listening and spoken language
development. Researchers are interested in discovering factors
related to improvements in educational outcomes within these programs.

The data set contains a suite of available predictors including:

- Gender (`male`)
- Number of household siblings (`siblings`)
- Index of family involvement (`family_inv`)
- Whether the primary household language is not English (`non_english`)
- Presence of a previous disability (`prev_disab`)
- Non-white race (`non_white`)
- Age at time of testing (in months, `age_test`)
- Whether hearing loss is not severe (`non_severe_hl`)
- Whether the subject's mother obtained a high school diploma or better (`mother_hs`)
- Whether the hearing impairment was identified by 3 months of age (`early_ident`)

The outcome variable is a standardized test score in one of several
learning domains.

In [ ]:
# Load the test scores to be analyzed.
test_scores = pd.read_csv(pm.get_data('test_scores.csv'), index_col=0)
test_scores.head()

In [ ]:
# Examine a histogram of the outcomes
test_scores['score'].hist()

plt.show()

In [ ]:
# Dropping missing values is, generally, a **very bad idea**. We do so
# in this case for simplicity. Additionally, we convert all values to
# floating point numbers.
X = test_scores.dropna().astype(float)

# The `DataFrame.pop()` method drops the specified item from the
# `DataFrame` and returns the dropped items.
y = X.pop('score')

# Standardize the features
X -= X.mean() # Centered at the mean
X /= X.std() # Normalize to the standard deviation

# I'm uncertain. What do `N` and `D` stand for?
N, D = X.shape
N, D

## The Model

This problem is a more realistic problem than our introductory
linear regression problem. Specifically,

- The problem is a **multivariate** regression problem
- We do not know, _a priori_, which parameters are
  relevant for constructing a statistical model.

A number of approaches are available for solving the second issue;
however, we will use _regularization_, a popular automated approach
that penalizes ineffective co-variates by shrinking them toward zero
if they do not contribute toward predicting outcomes.

In a Bayesian context, instead of using methods like lasso or
ridge regression, we apply an appropriate **prior** to the
regression coefficients. One such prior is the _hierarchical
regularized horseshoe_. This prior uses **two** regularization
strategies:

- One global
- A set of local parameters - one for each coefficient

The key to making this work is by selecting a [long-tailed
distribution](https://en.wikipedia.org/wiki/Long_tail)
as the shrinkage priors. This approach allows some priors(?)
to be non-zero while pushing the rest toward zero.

The horseshoe prior for each regression coefficient, $\beta_{i}$,
looks like:

$$
\beta_{i} \sim \mathcal{N} ( 0, \tau^{2} \cdot \widetilde{\lambda}_{i}^{2})
$$

where $\sigma$ is the prior on the error standard deviation that
will also be used for the model likelihood.

In this expression, $\tau$ is the global shrinkage parameter and
$\widetilde{\lambda}_{i}$ is the local shrinkage parameter.



Let's start global. For the prior on $\tau$, we will use the
Half-StudentT distribution. This choice is reasonable because
it is heavy tailed.

$$
\tau \sim Half-StudentT_{2}(\frac{D_{0}}{D - D_{0}} \cdot \frac{\sigma}{\sqrt N})
$$

One catch: our parameterization of the prior requires a constant,
$D_{0}$, which represents the true number of non-zero parameters.
Fortunately, we only need a reasonable guess and it only need to
be within an order of magnitude of the true number. We'll use
half the number of predictors as our guess:
```python
D0 = int(D / 2)
```

The local shrinkage parameters are defined by the ratio

$$
\widetilde{\lambda}_{i}^{2} = \frac{c^{2} \lambda_{i}^{2}}{c^{2} + \tau^{2} \lambda_{i}^{2}}
$$

To complete this specification, we need priors on
$\lambda_{i}$ and $c$. Similar to the global shrinkage, we use
a long-tailed Half-StudentT on the $\lambda_{i}$. We need $c^{2}$
to be strictly positive but not necessarily long-tailed.
Consequently, we will use an inverse gamma prior on $c^{2}$
where $c^{2} \sim InverseGamma(1, 1)$.

Finally, to allow the NUTS sampler to sample the $\beta_{i}$
more efficiently, we will **re-parameterize** as follows:

$$
\begin{gather}
z_{i} \sim \mathcal{N}(0, 1) \\
\beta_{i} = z_{i} \cdot \tau \cdot \widetilde{\lambda}_{i}
\end{gather}
$$

You will often encounter this re-parameterization in practice.